In [1]:
import cv2

In [2]:
img2 = cv2.imread("./data/images/train/{}".format('BloodImage_00000.jpg'))

In [3]:
import torch

In [4]:
import os

In [5]:
file_list = os.listdir('./data/images/train')

In [6]:
import config

In [7]:
len(file_list)

364

In [8]:
x_train = torch.zeros((len(file_list), config.img_size_H, config.img_size_W, 3))

In [9]:
for idx ,file_name in enumerate(file_list):
    x_train[idx] = torch.from_numpy(cv2.imread("./data/images/train/{}".format(file_name)))


In [10]:
x_train = x_train.permute(0, 3, 1, 2)/255

In [11]:
x_train[1][1][1][1]

tensor(0.6078)

In [12]:
x_train.shape

torch.Size([364, 3, 480, 640])

In [13]:
y_train = torch.zeros((len(file_list), config.out_W, config.out_H, config.nclass + 5))

In [14]:
y_train.shape

torch.Size([364, 10, 8, 8])

In [15]:
file_list_label = os.listdir('./data/labels/train')

In [16]:
with open("./data/labels/train/{}".format("BloodImage_00000.txt"), 'r') as file:
    # Đọc nội dung của tệp và gán nó cho một biến
    file_content = file.read()

In [17]:
y_train = torch.zeros((len(file_list), config.out_W, config.out_H, config.nclass + 5))
for idx ,file_name in enumerate(file_list_label):
    with open("./data/labels/train/{}".format(file_name), 'r') as file:
    # Đọc nội dung của tệp và gán nó cho một biến
        file_content = file.read()
        lines = file_content.splitlines()
        for line in lines:
            values = line.split()
            class_label, x_center_norm, y_center_norm, width_norm, height_norm = map(float, values)
            cl = torch.zeros((3))
            cl[int(class_label)] = 1
            # x_center, y_center, w, h = int(x_center_norm * config.img_size_W),  int(y_center_norm * config.img_size_H), int(width_norm * config.img_size_W), int(height_norm * config.img_size_H)
            # index của object trên ma trận ô vuông 10x8
            x_idx, y_idx = int(x_center_norm * config.out_W), int(y_center_norm * config.out_H)
            y_tensor = torch.zeros(config.nclass + 5)
            y_tensor[0] = 1
            y_tensor[1] = x_center_norm
            y_tensor[2] = y_center_norm
            y_tensor[3] = width_norm
            y_tensor[4] = height_norm
            y_tensor[5:] = cl
            y_train[idx][x_idx][y_idx] = y_tensor

In [18]:
x_train.shape

torch.Size([364, 3, 480, 640])

In [19]:
x = torch.rand((2, 10, 8, 13))

In [20]:
x[0][6][1]

tensor([0.3478, 0.7958, 0.0775, 0.1170, 0.8328, 0.0117, 0.8483, 0.3602, 0.7805,
        0.9589, 0.2253, 0.0167, 0.0509])

In [21]:
y_train[0][6][1]

tensor([1.0000, 0.6781, 0.1271, 0.1453, 0.2083, 0.0000, 1.0000, 0.0000])

In [1]:
import dataset

In [2]:
X, y = dataset.load_data_to_train()

In [3]:
X.shape

torch.Size([364, 3, 480, 640])

In [4]:
y.shape

torch.Size([364, 10, 8, 8])

In [13]:
from torch.utils.data import TensorDataset, DataLoader
import torch

In [8]:
LEARNING_RATE = 2e-5
DEVICE = "cuda" if torch.cuda.is_available else "cpu"
BATCH_SIZE = 16 # 64 in original paper but I don't have that much vram, grad accum?
WEIGHT_DECAY = 0
EPOCHS = 1000
NUM_WORKERS = 2
PIN_MEMORY = True
LOAD_MODEL = False
LOAD_MODEL_FILE = "overfit.pth.tar"
IMG_DIR = "data/images"
LABEL_DIR = "data/labels"

In [15]:
x_train = torch.Tensor(X)
y_train = torch.Tensor(y)

# Tạo một TensorDataset từ x_train và y_train
dataset = TensorDataset(x_train, y_train)

In [16]:
train_loader = DataLoader(
    dataset=dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
    shuffle=True,
    drop_last=True
)

In [17]:
from tqdm import tqdm

In [18]:
loop = tqdm(train_loader, leave=True)

  0%|          | 0/22 [00:00<?, ?it/s]

In [19]:
for batch_idx, (x, y) in enumerate(loop):
    print(x.shape, y.shape)

 23%|██▎       | 5/22 [00:35<01:30,  5.30s/it]

torch.Size([16, 3, 480, 640]) torch.Size([16, 10, 8, 8])
torch.Size([16, 3, 480, 640]) torch.Size([16, 10, 8, 8])
torch.Size([16, 3, 480, 640]) torch.Size([16, 10, 8, 8])
torch.Size([16, 3, 480, 640]) torch.Size([16, 10, 8, 8])
torch.Size([16, 3, 480, 640]) torch.Size([16, 10, 8, 8])
torch.Size([16, 3, 480, 640]) torch.Size([16, 10, 8, 8])
torch.Size([16, 3, 480, 640]) torch.Size([16, 10, 8, 8])
torch.Size([16, 3, 480, 640]) torch.Size([16, 10, 8, 8])


 59%|█████▉    | 13/22 [00:35<00:12,  1.36s/it]

torch.Size([16, 3, 480, 640]) torch.Size([16, 10, 8, 8])
torch.Size([16, 3, 480, 640]) torch.Size([16, 10, 8, 8])
torch.Size([16, 3, 480, 640]) torch.Size([16, 10, 8, 8])
torch.Size([16, 3, 480, 640]) torch.Size([16, 10, 8, 8])
torch.Size([16, 3, 480, 640]) torch.Size([16, 10, 8, 8])
torch.Size([16, 3, 480, 640]) torch.Size([16, 10, 8, 8])
torch.Size([16, 3, 480, 640]) torch.Size([16, 10, 8, 8])
torch.Size([16, 3, 480, 640]) torch.Size([16, 10, 8, 8])


 95%|█████████▌| 21/22 [00:36<00:00,  1.78it/s]

torch.Size([16, 3, 480, 640]) torch.Size([16, 10, 8, 8])
torch.Size([16, 3, 480, 640]) torch.Size([16, 10, 8, 8])
torch.Size([16, 3, 480, 640]) torch.Size([16, 10, 8, 8])
torch.Size([16, 3, 480, 640]) torch.Size([16, 10, 8, 8])
torch.Size([16, 3, 480, 640]) torch.Size([16, 10, 8, 8])
torch.Size([16, 3, 480, 640]) torch.Size([16, 10, 8, 8])


100%|██████████| 22/22 [00:36<00:00,  1.66s/it]
